# Numerical verification of some discrete boundary potential formulas
This notebook is supplementary material for 

"_A potential theory on weighted graphs_" by
Fernando Guevara Vasquez and Trent DeGiovanni

The idea here is to generate a random graph subdivided into boundary nodes $B$, exterior nodes $\Omega^+$ and a region of interest $\Omega^-$ that is connected to the rest of the nodes via $\partial\Omega$. Then we test the different identities on appropriate solutions to the Dirichlet problem on a graph. All the tests here work by computing the difference between the left and right hand side of a particular equality and checking that we numerically a vector close to zero using the `@test` macro. If all the tests in a cell pass, then `Test Passed` is displayed in the cell output.

In this code we use `Ωe` for exterior and `Ωi` for interior (and similarly for the other quantities that depend on interior vs exterior)}


## Graph construction

In [2]:
using LinearAlgebra, SparseArrays, Test

nB = 5; nΩe = 10; n∂Ω = 5; nΩi = 10 # select number of nodes

nV  = nB + nΩe + n∂Ω + nΩi
# define sets of nodes as indices
Ωi = 1:nΩi
∂Ω = nΩi .+ (1:n∂Ω)
Ωe = (nΩi + n∂Ω) .+ (1:nΩe)
B = (nΩi + n∂Ω + nΩe) .+ (1:nB)
V = 1:nV # all vertices
Vᵒ = Ωe ∪ ∂Ω ∪ Ωi # all the vertices minus the ones boundary conditions are imposed
println("""The sets are:
    Ωi  = $Ωi
    ∂Ω = $∂Ω
    Ωe  = $Ωe
    B  = $B
    V  = $V
    Vᵒ = $Vᵒ""")

# generate incidence matrix with conductivities that are uniform random (0,1)
# using Erdös-Renyi model
p = 0.5 # probability of getting a link between two nodes
A = spzeros(nV,nV)
for i=1:nV, j=1:i-1
    A[i,j] =  rand()*(rand()>p)
end
A = -(A + A')/2

# get rid of connections between Ωe and Ωi
A[Ωe,Ωi] .= 0; A[Ωi,Ωe] .= 0

# get rid of connections between B and anything other than Ωe
A[B,∂Ω ∪ Ωi] .= 0; A[∂Ω ∪ Ωi,B] .= 0

# check ∂Ω is connected to both Ωi and Ωe
@assert sum(A[∂Ω,Ωi])<0 && sum(A[∂Ω,Ωe])<0 # if error is raised, simply rerun cell to draw another graph

The sets are:
Ωi  = 1:10
∂Ω = 11:15
Ωe  = 16:25
B  = 26:30
V  = 1:30
Vᵒ = [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 11, 12, 13, 14, 15, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


## Laplacian and Green functions
Note the identities that we find are not affected by the sign of the Laplacian (i.e. whether we choose it to be a positive semidefinite or negative semidefinite matrix)

In [3]:
# ensures a matrix has zero row sum by adjusting diagonal elements
zerosum(A) = A - spdiagm(A*ones(size(A,1))) 
L = zerosum(A)  # negative semi-definite Laplacian
#L = -zerosum(A) # positive semi-definite graph Laplacian

# Green functions with 0 Dirichlet condition on B
G = zeros(nV,nV)
G[Vᵒ,Vᵒ] = inv(Matrix(L[Vᵒ,Vᵒ]));

## The exterior and interior Laplacians
The edge partition of the unity for conductivities we chose is of the form $p^-|_{\mathcal{E}^{\partial\Omega}}=\alpha$ and $p^+|_{\mathcal{E}^{\partial\Omega}}=1-\alpha$ for $\alpha \in [0,1]$. The exterior $L^+$ and interior $L^-$ Laplacians should satisfy $L=L^+ + L^-$.

In [4]:
α = 0.3 # blending factor (anything in [0,1])

# exterior Laplacian (zero inside Ω)
Le = copy(L);
Le[Ωi,:] .= 0; Le[:,Ωi] .= 0
Le[∂Ω,∂Ω] .*= 1-α
Le = zerosum(Le);

# interior Laplacian (zero outside Ω, i.e. B∪E)
Li = copy(L);
Li[B∪Ωe,:] .= 0; Li[:,B∪Ωe] .=0 ;
Li[∂Ω,∂Ω] .*= α
Li = zerosum(Li);
@test norm(Li+Le-L) ≈ 0 atol=1e-10

Test Passed

## Trace operators and single, double layer potential operators
Here we define:
* Dirichlet trace operator $\gamma_0$ and the Neumann trace operators $\gamma_1^\pm$
* Single $\mathcal{S}$ and double layer $\mathcal{D}^\pm$ potential operators
* Indicator function $\mathbb{1}$ of a set

In [5]:
R = I(nV)[∂Ω,:] # restriction to ∂Ω
γ0 = R          # Dirichlet trace operator
γ1e = - R*Le    # exterior Neumann trace operator
γ1i =   R*Li    # interior Neumann trace operator

S  = G*γ0'      # single layer potential operator
De = G*γ1e'     # exterior double layer potential operator
Di = G*γ1i'     # interior double layer potential operator

com(C,D) = C*D - D*C        # matrix commutator

𝟙(X) = [ i ∈ X for i ∈ V];  # indicator function of the set X

𝟙 (generic function with 1 method)

## Interior reproduction formulas

In [6]:
# generate a u that solves the Dirichlet problem inside ∂Ω ∪ Ωi
u = zeros(nV)
u[B] = randn(nB)
u[Vᵒ] = -L[Vᵒ,Vᵒ]\(L[Vᵒ,B]*u[B])

# check interior reproduction formulas:
@test norm(S*γ1e*u - De*γ0*u - u.*𝟙(∂Ω ∪ Ωi)) ≈ 0 atol=1e-10
@test norm(S*γ1i*u - Di*γ0*u - u.*𝟙(Ωi))      ≈ 0 atol=1e-10

# check formulas with commutators:
@test norm(( G*com(Le,R'*R)*u - u)[∂Ω ∪ Ωi]) ≈ 0 atol=1e-10
@test norm((-G*com(Li,R'*R)*u - u)[Ωi])      ≈ 0 atol=1e-10

Test Passed

## Continuity of flux for a harmonic function

In [7]:
@test norm((γ1e - γ1i)*u) ≈ 0 atol=1e-10

Test Passed

## Exterior reproduction formulas

In [8]:
# generate a v that solves teh Dirichlet problem outside  ∂Ω ∪ Ω
f = zeros(nV); f[Ωi] = randn(nΩi); # forcing term
v = zeros(nV);
v[Vᵒ]=-L[Vᵒ,Vᵒ]\f[Vᵒ];

# check exterior reproduction formulas:
@test norm(-S*γ1e*v + De*γ0*v - v.*𝟙(Ωe))      ≈ 0 atol=1e-10
@test norm(-S*γ1i*v + Di*γ0*v - v.*𝟙(Ωe∪∂Ω))   ≈ 0 atol=1e-10

# check formulas with commutators:
@test norm((-G*com(Le,R'*R)*v - v)[Ωe])        ≈ 0 atol=1e-10
@test norm(( G*com(Li,R'*R)*v - v)[Ωe∪∂Ω])     ≈ 0 atol=1e-10

Test Passed

## Jump relations

In [9]:
@test norm((γ1e-γ1i)*S  + I) ≈ 0 atol=1e-10  # discontinuity of Neumann trace of single layer potential
@test norm(γ0*(De - Di) + I) ≈ 0 atol=1e-10  # discontinuity of Dirichlet trace of double layer potential
@test norm(γ1e*Di - γ1i*De)  ≈ 0 atol=1e-10  # continuity of Neumann trace of double layer potential

Test Passed

## Boundary Layer Operators

In [10]:
bS = γ0*S          # Single Layer Operator
bD = γ0*(De+Di)/2   # Double Layer Operator
bDa = (γ1e+γ1i)*S/2 # Adjoint Double Layer Operator
bH = -γ1e*Di;        # Hypersingular operator

## Jump relations for boundary layer operators

In [11]:
# discontinuity of Neumann trace of single layer potential
@test norm(γ1e*S - ( -I/2 + bD' )) ≈ 0 atol=1e-10
@test norm(γ1i*S - (  I/2 + bD' )) ≈ 0 atol=1e-10

# discontinuity of Dirichlet trace of double layer potential
@test norm(γ0*De - ( -I/2 + bD  )) ≈ 0 atol=1e-10
@test norm(γ0*Di - (  I/2 + bD  )) ≈ 0 atol=1e-10

Test Passed

## Calderón projectors

In [12]:
C = [-bD     bS
     bH     bD' ]
Pi = I/2 + C # interior Calderón projector
Pe = I/2 - C # exterior Calderón projector
@test norm(Pi*Pi - Pi) ≈ 0 atol=1e-10
@test norm(Pe*Pe - Pe) ≈ 0 atol=1e-10

Test Passed

## Identities between boundary layer operators
\begin{align}
 S H &= -D^2 + \frac{I}{4},\\
 H S &= -(D')^2 + \frac{I}{4},\\
 S D' &= D S,\\
 D' H &= H D.
\end{align}

In [13]:
@test norm(bS*bH - (-bD^2+I/4)) ≈ 0 atol=1e-10
@test norm(bH*bS - (-(bD')^2+I/4)) ≈ 0 atol=1e-10
@test norm(bS*bD' - bD*bS) ≈ 0 atol=1e-10
@test norm(bD'*bH - bH*bD) ≈ 0 atol=1e-10

Test Passed

## DtN map tests
Checks DtN map formula $\Lambda = (\frac{I}{2} + D') S^{-1} = S^{-1} (\frac{I}{2} + D) = H + (\frac{I}{2} + D') S^{-1} (\frac{I}{2} + D)$.

In [21]:
# interior
DtN1 = Li[∂Ω,∂Ω] - Li[∂Ω,Ωi]*(Li[Ωi,Ωi]\Array(Li[Ωi,∂Ω]))
DtN2 = (I/2+bD')*inv(bS) 
DtN3 = inv(bS)*(I/2+bD)
DtN4 = bH + (I/2+bD')*inv(bS)*(I/2+bD)
@test norm(DtN1 - DtN2) ≈ 0 atol=1e-10
@test norm(DtN1 - DtN3) ≈ 0 atol=1e-10
@test norm(DtN1 - DtN4) ≈ 0 atol=1e-10

# exterior
DtN1 = -(Le[∂Ω,∂Ω] - Le[∂Ω,Ωe]*(Le[Ωe,Ωe]\Array(Le[Ωe,∂Ω])))
DtN2 = (-I/2+bD')*inv(bS) 
DtN3 = inv(bS)*(-I/2+bD)
DtN4 = -bH -(-I/2+bD')*inv(bS)*(-I/2+bD)
@test norm(DtN1 - DtN2) ≈ 0 atol=1e-10
@test norm(DtN1 - DtN3) ≈ 0 atol=1e-10
@test norm(DtN1 - DtN4) ≈ 0 atol=1e-10

Test Passed

In [17]:
Dt1

5×5 Matrix{Float64}:
 -0.916003    0.0668366   0.113667   0.359822    0.0755227
  0.0668366  -1.57501     0.419837   0.0689125   0.0987656
  0.113667    0.419837   -1.84274    0.348351    0.289487
  0.359822    0.0689125   0.348351  -1.76443     0.454736
  0.0755227   0.0987656   0.289487   0.454736   -1.59234

## Check invertibility of operators
These are based on the uniqueness of certain problems

In [148]:
@test rank(bS) == nB # BSO should be invertible (uniqueness Dir prob)
@test rank(-I/2 + bD) == nB # uniqueness exterior Neumann prob
@test rank(I/2 + bD) == nB-1 # interior Neumann prob up to constants
@test norm((I/2 + bD)*ones(n∂Ω)) ≈ 0 atol=1e-10

Test Passed

## Spectrum of Neumann-Poincaré operator inside [-1/2,1/2]

In [149]:
@test all(abs.(eigvals(bD')) .<= 1/2+1e-10)

Test Passed